In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from datetime import datetime
from PIL import features
import numpy as np

import sys
import os


current_dir = os.getcwd()
sys.path.append(os.path.join(current_dir, '..'))

import ETL.utils as ut

In [28]:
# Load datasets

steam_games_df = pd.read_csv('../data/generated/steam_games.csv',dtype={'id': str})
items_df = pd.read_csv('../data/generated/items.csv', dtype={'item_id': str})
reviews_df = pd.read_csv('../data/generated/reviews_sentiment.csv', dtype={'item_id': str})


In [29]:
columnas_object = steam_games_df.select_dtypes(include=['object']).columns
steam_games_df[columnas_object] = steam_games_df[columnas_object].astype(str)

columnas_object = items_df.select_dtypes(include=['object']).columns
items_df[columnas_object] = items_df[columnas_object].astype(str)

columnas_object = reviews_df.select_dtypes(include=['object']).columns
reviews_df[columnas_object] = reviews_df[columnas_object].astype(str)

### Creation of specific dataframes for the queries."

#### PlayTimeGenre

In [30]:
# Aux function
def parse_date(date_str):
    if date_str == 'Not specified':
        return np.nan  # or any other placeholder value you prefer
    try:
        return datetime.strptime(date_str, '%Y-%m-%d')
    except ValueError:
        try:
            return datetime.strptime(date_str, '%b %Y')
        except ValueError:
            return np.nan  # or any other placeholder value

In [31]:
# Aux function
def process_genres(row):
    if isinstance(row['genres'], list):
        return row['genres']
    elif isinstance(row['genres'], str):
        return [genre.strip(" '[]") for genre in row['genres'].split(',')]
    else:
        return [row['genres']]

In [32]:

# Step 1: Merge DataFrames based on the "id" and "item_id" keys
merged_df = pd.merge(steam_games_df, items_df, left_on='id', right_on='item_id', how='inner')

# Step 2: Explore genre lists and expand them into rows
merged_df = merged_df.explode('genres')

# Step 3: Apply the parse_date function to the 'release_date' column
merged_df['release_date'] = merged_df['release_date'].apply(parse_date)

# Step 4: Filter rows with positive playtime
merged_df = merged_df[merged_df['playtime_forever'] > 0]

# Apply the function to each row
merged_df['genres'] = merged_df.apply(process_genres, axis=1)

# Step 6: Explode the DataFrame again to have one row for each genre
merged_df = merged_df.explode('genres')

# Step 7: Group by genre and year, and calculate the maximum sum of playtime
play_time_genre_df = merged_df.groupby(['genres', merged_df['release_date'].dt.year])['playtime_forever'].max().reset_index()

# Step 8: Rename columns as per your requirements
play_time_genre_df.columns = ['genre', 'year', 'max_playtime_hours']
play_time_genre_df["year"] = play_time_genre_df["year"].astype(int)

In [33]:
ut.data_overview(play_time_genre_df)


Total rows:  189

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genre,[<class 'str'>],100.0,189,0.0,0
1,year,[<class 'int'>],100.0,189,0.0,0
2,max_playtime_hours,[<class 'float'>],100.0,189,0.0,0


In [34]:
play_time_genre_df['genre']

0                action
1                action
2                action
3                action
4                action
             ...       
184            strategy
185            strategy
186           utilities
187           utilities
188    video production
Name: genre, Length: 189, dtype: object

In [35]:
path = r'../simplified-data/'
play_time_genre_df.to_csv(path + 'play_time_genre.csv', index=False)

#### UserFoGenre

In [36]:
# Step 1: Merge DataFrames based on the "id" and "item_id" keys
merged_df = pd.merge(steam_games_df, items_df, left_on='id', right_on='item_id', how='inner')

# Step 2: Explore genre lists and expand them into rows
merged_df = merged_df.explode('genres')

merged_df['release_date'] = merged_df['release_date'].apply(parse_date)

# Step 3: Drop rows with null values in relevant columns
merged_df = merged_df.dropna(subset=['playtime_forever', 'genres', 'user_id', 'release_date'])

# Step 4: Filter rows with positive playtime
merged_df = merged_df[merged_df['playtime_forever'] > 0]

# Apply the function to each row
merged_df['genres'] = merged_df.apply(process_genres, axis=1)

# Apply the function to each row and create a new column 'processed_genres'
merged_df['processed_genres'] = merged_df.apply(process_genres, axis=1)

# Explode the 'processed_genres' column to create separate rows for each genre
merged_df = merged_df.explode('processed_genres')

# Step 5: Group by processed_genres, user, year, and calculate the sum of playtime
user_for_genre_df = merged_df.groupby(['processed_genres', 'user_id', merged_df['release_date'].dt.year.astype(int)])['playtime_forever'].sum().reset_index()

# Step 6: Find the user with the most playtime for each genre and year
user_for_genre_df = user_for_genre_df.loc[user_for_genre_df.groupby(['processed_genres', 'release_date'])['playtime_forever'].idxmax()]

# Rename columns as per your requirements
user_for_genre_df.columns = ['genres', 'user', 'year', 'playtime_forever']
user_for_genre_df["playtime_forever"] = user_for_genre_df["playtime_forever"].astype(int)

In [37]:
ut.data_overview(user_for_genre_df)


Total rows:  189

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,genres,[<class 'str'>],100.0,189,0.0,0
1,user,[<class 'str'>],100.0,189,0.0,0
2,year,[<class 'int'>],100.0,189,0.0,0
3,playtime_forever,[<class 'int'>],100.0,189,0.0,0


In [38]:
user_for_genre_df['genres']

5311               action
8419               action
11800              action
16247              action
6400               action
               ...       
86713            strategy
86766            strategy
87520           utilities
87519           utilities
87522    video production
Name: genres, Length: 189, dtype: object

In [39]:
user_for_genre_df.iloc[0]

genres                         action
user                76561198041356854
year                             1990
playtime_forever                 1424
Name: 5311, dtype: object

In [40]:
path = r'../simplified-data/'
user_for_genre_df.to_csv(path + 'user_for_genre.csv', index=False)

#### UsersRecommend

In [42]:
# Filter only positive/neutral recommendations in reviews_df
filtered_reviews_df = reviews_df[(reviews_df['recommend'] == True) & (reviews_df['sentiment_analysis'].isin([1, 2]))]

# Merge dataframes using the filtered dataframe
merged_df = pd.merge(filtered_reviews_df, items_df, on='item_id')

# Convert the 'posted' column to datetime type
merged_df['posted'] = pd.to_datetime(merged_df['posted'], errors='coerce')

# Extract the year from the 'posted' column
merged_df['year'] = merged_df['posted'].dt.year.where(merged_df['posted'].notnull(), 'Year not specified')

# Get the top 3 most recommended games per year using nlargest
top3_df = merged_df.groupby(['year', 'item_id']).size().groupby('year', group_keys=False).nlargest(3).reset_index(name='recommendations_count')



In [43]:
ut.data_overview(top3_df)


Total rows:  17

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,year,[<class 'int'>],100.0,17,0.0,0
1,item_id,[<class 'str'>],100.0,17,0.0,0
2,recommendations_count,[<class 'int'>],100.0,17,0.0,0


In [44]:
top3_df.head

<bound method NDFrame.head of     year item_id  recommendations_count
0   2010   22380                     66
1   2010   17450                     13
2   2011    4000                  76200
3   2011     550                   8680
4   2011  105600                   2838
5   2012    4000                 133350
6   2012     240                  40772
7   2012     220                  16944
8   2013    4000                1924050
9   2013     550                  92008
10  2013     240                  71351
11  2014    4000                5410200
12  2014     240                 417913
13  2014     730                 298309
14  2015    4000                2133600
15  2015     730                 424672
16  2015     240                 173281>

In [45]:
top3_df['year'] = pd.to_numeric(top3_df['year'], errors='coerce').astype('Int64')

In [46]:
top3_df = top3_df.dropna(subset=['year'])
top3_df.head

<bound method NDFrame.head of     year item_id  recommendations_count
0   2010   22380                     66
1   2010   17450                     13
2   2011    4000                  76200
3   2011     550                   8680
4   2011  105600                   2838
5   2012    4000                 133350
6   2012     240                  40772
7   2012     220                  16944
8   2013    4000                1924050
9   2013     550                  92008
10  2013     240                  71351
11  2014    4000                5410200
12  2014     240                 417913
13  2014     730                 298309
14  2015    4000                2133600
15  2015     730                 424672
16  2015     240                 173281>

In [47]:
path = r'../simplified-data/'
top3_df.to_csv(path + 'users_recommend.csv', index=False)

#### UsersWorstDeveloper

In [52]:
# Filter reviews
filtered_reviews_df = reviews_df[(reviews_df['recommend'] == False) & (reviews_df['sentiment_analysis'] == 0)]

# Merge dataframes using the 'item_id' column
merged_df = pd.merge(filtered_reviews_df, items_df, on='item_id')

# Add the 'developer' column from steam_games_df
merged_df = pd.merge(merged_df, steam_games_df[['id', 'developer']], left_on='item_id', right_on='id', how='left')

# Convert the 'posted' column to datetime and extract the year
merged_df['posted'] = pd.to_datetime(merged_df['posted'], errors='coerce')
merged_df['year'] = merged_df['posted'].dt.year.where(merged_df['posted'].notnull(), 'Year not specified')

# Filter non-recommended negative reviews with sentiment analysis equal to 0
negative_reviews_df = merged_df[(merged_df['recommend'] == False) & (merged_df['sentiment_analysis'] == 0)]

# Ensure that the year is an integer
negative_reviews_df['year'] = negative_reviews_df['year'].astype('Int64')

# Get the top 3 developers with the three least recommended games per year
top3_least_recommended_by_year = negative_reviews_df.groupby(['year', 'developer', 'item_id']).size().groupby(['year', 'developer'], group_keys=False).nsmallest(3).reset_index(name='least_recommended_count')


In [53]:
ut.data_overview(top3_least_recommended_by_year)


Total rows:  221

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,year,[<class 'int'>],100.0,221,0.0,0
1,developer,[<class 'str'>],100.0,221,0.0,0
2,item_id,[<class 'str'>],100.0,221,0.0,0
3,least_recommended_count,[<class 'int'>],100.0,221,0.0,0


In [54]:
top3_least_recommended_by_year

,year,developer,item_id,least_recommended_count
0,2011,ubisoft montpellier,33460,3
1,2013,avalanche studios,8190,151
2,2013,bohemia interactive,221100,16
3,2013,chucklefish,211820,11
4,2013,creative assembly,214950,18
...,...,...,...,...
216,2015,valve,620,438
217,2015,valve,20,756
218,2015,valve,730,17887
219,2015,wild shadow studios,200210,213


#### sentiment_analysis

In [55]:
# Combinar DataFrames
merged_df = pd.merge(steam_games_df, reviews_df, left_on='id', right_on='item_id')

# Crear columnas para cada categoría de sentimiento
sentiment_columns = ['Negative', 'Neutral', 'Positive']
for sentiment in sentiment_columns:
    merged_df[sentiment] = (merged_df['sentiment_analysis'] == sentiment_columns.index(sentiment)).astype(int)

# Agrupar por desarrollador y sumar los conteos
sentiment_analysis_df = merged_df.groupby('developer')[sentiment_columns].sum().reset_index()

In [56]:
ut.data_overview(sentiment_analysis_df)


Total rows:  1172

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,developer,[<class 'str'>],100.0,1172,0.0,0
1,Negative,[<class 'int'>],100.0,1172,0.0,0
2,Neutral,[<class 'int'>],100.0,1172,0.0,0
3,Positive,[<class 'int'>],100.0,1172,0.0,0


In [57]:
path = r'../simplified-data/'
sentiment_analysis_df.to_csv(path + 'sentiment_analysis.csv', index=False)

#### recomendacion_juego

In [58]:
desired_columns = ["genres", "tags", "specs","id","app_name"]
game_recomendation_df = steam_games_df[desired_columns].copy()

In [59]:
path = r'../simplified-data/'
game_recomendation_df.to_csv(path + 'game_recomendation.csv', index=False)